**Predicting Commuter Transportation Choices (Python)**

In [45]:
# Importamos paquetes
from __future__ import division, print_function
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [17]:
# importamos datos 
from google.colab import files
sydney = files.upload()
print('/n', sydney)

Saving sydney.csv to sydney (2).csv
/n {'sydney.csv': b'cartime,carcost,traintime,traincost,choice\r70,50,64,39,TRAIN\r50,230,60,32,TRAIN\r50,70,58,40,CAR\r60,108,93,62,CAR\r70,60,68,26,TRAIN\r20,32,72,65,CAR\r40,30,60,37,TRAIN\r15,8,22,20,CAR\r60,60,83,31,TRAIN\r29,20,40,17,CAR\r36,30,65,70,CAR\r50,30,125,60,CAR\r63,35,145,60,CAR\r20,30,48,25,CAR\r60,40,85,45,CAR\r30,20,60,35,TRAIN\r47,60,79,26,TRAIN\r60,72,50,26,TRAIN\r55,40,43,26,TRAIN\r35,80,51,29,TRAIN\r15,30,25,20,TRAIN\r55,50,70,27,TRAIN\r50,50,65,26,TRAIN\r39,25,50,55,CAR\r17,20,78,35,CAR\r35,56,87,40,CAR\r65,55,90,26,TRAIN\r38,40,74,23,TRAIN\r15,15,40,34,CAR\r65,130,74,30,TRAIN\r40,80,80,88,CAR\r15,35,45,33,TRAIN\r55,82,65,36,TRAIN\r25,36,68,36,TRAIN\r75,72,69,65,CAR\r25,60,44,18,TRAIN\r61,50,61,33,TRAIN\r15,36,37,28,CAR\r15,44,35,50,CAR\r35,36,68,25,CAR\r50,60,73,30,CAR\r65,120,47,46,TRAIN\r40,60,67,37,CAR\r60,56,75,26,CAR\r35,56,71,45,CAR\r20,24,83,55,CAR\r31,25,64,45,CAR\r35,50,80,49,CAR\r34,77,70,60,CAR\r65,50,90,30,TRAIN\

In [18]:
sydney = pd.read_csv("sydney.csv")

In [19]:
# creamos listas que permitan pasar de una varaible tipo string a una tipo binaria o dummy
response_to_binary = {'TRAIN':1, 'CAR':0}

#la funcion "map()" ejecuta una función sobre cada uno de los elementos de un iterador 
#(generalmente una lista o tupla) y retorna un nuevo iterador cuyos elementos 
#son el resultado de dicha operación.
y = sydney['choice'].map(response_to_binary)
y

0      1
1      1
2      0
3      0
4      1
      ..
328    0
329    0
330    0
331    0
332    0
Name: choice, Length: 333, dtype: int64

In [22]:
#creamos objetos individuales con las columnas de las variables
cartime = sydney['cartime']
carcost = sydney['carcost']
traintime = sydney['traintime']
traincost = sydney['traincost']

0      70
1      50
2      50
3      60
4      70
       ..
328    27
329    25
330    50
331    25
332    35
Name: cartime, Length: 333, dtype: int64

In [25]:
# definimos una matriz para el predictor lineal

#creamos un array del mismo tamaño de "y" y completada con "1"s
Intercept = np.array([1] * len(y))

#creamos una variable x con las varaibles rpredictoras e "Intercept"
x = np.array([Intercept, cartime, carcost, traintime, traincost]).T
print(x)

[[  1  70  50  64  39]
 [  1  50 230  60  32]
 [  1  50  70  58  40]
 ...
 [  1  50  50  80  50]
 [  1  25  25  39  20]
 [  1  35  64  95  40]]


In [26]:
# modelo linear generalizado para  regresión logistica}
logistic_regression = sm.GLM(y, x, family=sm.families.Binomial())
#la función GLM de sm funciona igual que R


In [27]:
#creamos el ajuste del modelo e imprimimos el resultado
sydney_fit = logistic_regression.fit()
print(sydney_fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 choice   No. Observations:                  333
Model:                            GLM   Df Residuals:                      328
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -136.32
Date:                Wed, 17 Mar 2021   Deviance:                       272.63
Time:                        13:39:46   Pearson chi2:                     326.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4440      0.585     -2.468      0.0

In [30]:
#creamos la varaible "train_prob" que indicara la redicion de uso del tren
sydney['train_prob'] = sydney_fit.predict(linear = False) #agregamos el falso para que sea regresion logistica
sydney['train_prob']

0      0.649733
1      0.996237
2      0.470977
3      0.414966
4      0.918547
         ...   
328    0.108880
329    0.000710
330    0.182752
331    0.283052
332    0.356422
Name: train_prob, Length: 333, dtype: float64

In [36]:
# función para convertir la probabilidad en una predicción de la elección que ahcen sobre el tren vs auto
# si la respuesta esta por encima de "cutoff" sera train, caso contrario car.
def prob_to_response(response_prob, cutoff):
  if(response_prob > cutoff):
    return('TRAIN')
  else:
    return('CAR')

In [40]:
# añadimos predicciones binarias al data frame de sydney usando el valor de cutoff
sydney['choice_pred'] = \
sydney['train_prob'].apply(lambda d: prob_to_response(d, cutoff = 0.50))
# En lugar de usar def para definir nuestra función, hemos utilizado la palabra clave lambda; 
# a continuación escribimos x, y como argumentos de la función, y x + y como expresión.
sydney.head()

,cartime,carcost,traintime,traincost,choice,train_prob,choice_pred
0,70,50,64,39,TRAIN,0.649733,TRAIN
1,50,230,60,32,TRAIN,0.996237,TRAIN
2,50,70,58,40,CAR,0.470977,CAR
3,60,108,93,62,CAR,0.414966,CAR
4,70,60,68,26,TRAIN,0.918547,TRAIN


In [43]:
# evaluamos la performance del modelo de regresion logistica
# obtenemos una matriz de confusión y proporcion de la observaciones correctamente PREDICHAS
# de esa forma observamos que porcentaje de aceierto tenemos

#creamos tabla con la predicción y la elección real
# la funcion "crosstab" crea una tabal de frecuencias
cmat = pd.crosstab(sydney['choice_pred'], sydney['choice'])
cmat

choice,CAR,TRAIN
choice_pred,,
CAR,155,30
TRAIN,28,120


In [47]:
#la funcion ".ix[]" permite cortar la tabla segun nuesstras indicaciones
#funion de la mima forma que "loc" o "iloc". Utilizo iloc, es mas facil
a = float(cmat.iloc[0,0])
b = float(cmat.iloc[0,1])
c = float(cmat.iloc[1,0])
d = float(cmat.iloc[1,1])
n = a + b + c + d
print(n)
#la suma del total de registros

333.0


In [48]:
#calculamos el acierto del modelo
predictive_accuracy = (a + d)/n
print(cmat)
print('\n Percentage Correctly Predicted',\
round(predictive_accuracy, 3), "\n")

choice       CAR  TRAIN
choice_pred            
CAR          155     30
TRAIN         28    120

 Percentage Correctly Predicted 0.826 

